In [8]:
# see 'requirements.txt' for required modules

import requests
from pprint import pprint
import os
import pandas as pd
import time

from azure.ai.textanalytics import TextAnalyticsClient, TextAnalyticsApiKeyCredential

In [ ]:
import numpy as np
np.__version__

In [4]:
key = "f4a3533e6eb8431d9dca52beb055c8f5"
endpoint = "https://sotu.cognitiveservices.azure.com/"
new_variable = "test"
def authenticate_client(k_tx, e_tx):
    ta_credential = TextAnalyticsApiKeyCredential(k_tx)
    text_analytics_client = TextAnalyticsClient(
            endpoint=e_tx, credential=ta_credential)
    return text_analytics_client

client = authenticate_client(key, endpoint)

In [5]:
def sentiment_analysis_example(client, tf):

    document = []
    
    with open(tf, 'r') as f:
        for line in f:
            line = line.strip()
            line = line.replace(u'\xa0', u' ')
            document.append(line)
    f.close()

    response = client.analyze_sentiment(inputs=document)[0]
    return((response.sentiment, response.sentiment_scores.positive, response.sentiment_scores.negative))

In [23]:
# this program will expect the text files to be analyzed 
# to be located an a subdirectory named 'Text'


textdir = os.path.abspath("Bild") # os.path.abspath("Text")

returned_sentiment = []

with os.scandir(textdir) as entries:
    for entry in entries:
        if entry.is_file():
            entry_str = str(entry)
            trimmed_entry_str = entry_str[(entry_str.find("'")+1):entry_str.rfind("'")]
            entry_name = trimmed_entry_str[0:trimmed_entry_str.find('.')]
            entry_ext = trimmed_entry_str[trimmed_entry_str.find(".")+1:]
            if entry_ext.isnumeric() and 2012 <= int(entry_ext) <=2100:   # if we last that long
                aFile = textdir + "/" + entry_name + '.' + entry_ext
                (sae_sentiment,sae_pos,sae_neg) = sentiment_analysis_example(client, aFile)
                this_record = (entry_name + '.' + entry_ext, sae_sentiment, sae_pos, sae_neg)
                #this_record = (entry_name + '.' + entry_ext, sentiment_analysis_example(client, aFile))
                returned_sentiment.append(this_record)

print(returned_sentiment)

[('trump.2020', 'neutral', 0.02, 0.02), ('obama.2015', 'neutral', 0.13, 0.02), ('obama.2012', 'positive', 1.0, 0.0), ('trump.2017', 'neutral', 0.02, 0.01), ('obama.2014', 'neutral', 0.13, 0.02), ('trump.2019', 'neutral', 0.02, 0.01), ('obama.2013', 'positive', 0.79, 0.04), ('trump.2018', 'neutral', 0.07, 0.02), ('obama.2016', 'neutral', 0.13, 0.02)]


In [24]:
sae_df = pd.DataFrame(returned_sentiment, columns=['President.Year', 'Sentiment', 'Pos', 'Neg'])

In [25]:
sae_df.head()

,President.Year,Sentiment,Pos,Neg
0,trump.2020,neutral,0.02,0.02
1,obama.2015,neutral,0.13,0.02
2,obama.2012,positive,1.00,0.00
3,trump.2017,neutral,0.02,0.01
4,obama.2014,neutral,0.13,0.02
